# Chapter 6

## Multilayer perceptron regressor

In [1]:
from pyspark.sql import SparkSession 

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Intro") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/14 18:29:33 WARN Utils: Your hostname, Gabriels-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 10.40.1.32 instead (on interface en0)
25/10/14 18:29:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/14 18:29:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructField, StructType, StringType, DoubleType

custom_schema = StructType([
    StructField("Make", StringType(), True),
    StructField("Model", StringType(), True),
    StructField("Vehicle Class", StringType(), True),
    StructField("Cylinders", DoubleType(), True),
    StructField("Transmission", StringType(), True),
    StructField("Fuel Type", StringType(), True),
    StructField("Fuel Consumption City (L/100 km)", DoubleType(), True),
    StructField("Fuel Consumption Hwy (L/100 km)", DoubleType(), True),
    StructField("Fuel Consumption Comb (L/100 km)", DoubleType(), True),
    StructField("Fuel Consumption Comb (mpg)", DoubleType(), True),
    StructField("CO2", DoubleType(), True)])

In [4]:
co2_data = spark.read.format("csv")\
    .schema(custom_schema) \
    .option("header", True) \
    .load("./static/CO2_Emissions_Canada.csv")

In [5]:
co2_data.take(2)

25/10/14 18:30:10 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 11
CSV file: file:///Users/gafnts/Documents/Github/ml-with-spark/ml-with-spark/static/CO2_Emissions_Canada.csv


[Row(Make='ACURA', Model='ILX', Vehicle Class='COMPACT', Cylinders=2.0, Transmission='4', Fuel Type='AS5', Fuel Consumption City (L/100 km)=None, Fuel Consumption Hwy (L/100 km)=9.9, Fuel Consumption Comb (L/100 km)=6.7, Fuel Consumption Comb (mpg)=8.5, CO2=33.0),
 Row(Make='ACURA', Model='ILX', Vehicle Class='COMPACT', Cylinders=2.4, Transmission='4', Fuel Type='M6', Fuel Consumption City (L/100 km)=None, Fuel Consumption Hwy (L/100 km)=11.2, Fuel Consumption Comb (L/100 km)=7.7, Fuel Consumption Comb (mpg)=9.6, CO2=29.0)]

In [6]:
cols_only_continues_values = {'Fuel Consumption City (L/100 km)':0}

In [7]:
co2_data = co2_data.fillna(0.0)

In [8]:
co2_data.printSchema()

root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Vehicle Class: string (nullable = true)
 |-- Cylinders: double (nullable = false)
 |-- Transmission: string (nullable = true)
 |-- Fuel Type: string (nullable = true)
 |-- Fuel Consumption City (L/100 km): double (nullable = false)
 |-- Fuel Consumption Hwy (L/100 km): double (nullable = false)
 |-- Fuel Consumption Comb (L/100 km): double (nullable = false)
 |-- Fuel Consumption Comb (mpg): double (nullable = false)
 |-- CO2: double (nullable = false)



In [9]:
co2_data.take(2)

25/10/14 18:30:24 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 11
CSV file: file:///Users/gafnts/Documents/Github/ml-with-spark/ml-with-spark/static/CO2_Emissions_Canada.csv


[Row(Make='ACURA', Model='ILX', Vehicle Class='COMPACT', Cylinders=2.0, Transmission='4', Fuel Type='AS5', Fuel Consumption City (L/100 km)=0.0, Fuel Consumption Hwy (L/100 km)=9.9, Fuel Consumption Comb (L/100 km)=6.7, Fuel Consumption Comb (mpg)=8.5, CO2=33.0),
 Row(Make='ACURA', Model='ILX', Vehicle Class='COMPACT', Cylinders=2.4, Transmission='4', Fuel Type='M6', Fuel Consumption City (L/100 km)=0.0, Fuel Consumption Hwy (L/100 km)=11.2, Fuel Consumption Comb (L/100 km)=7.7, Fuel Consumption Comb (mpg)=9.6, CO2=29.0)]

# Prep the data for regression

turn the feature columns into one indexed column:

In [10]:
from pyspark.ml.feature import FeatureHasher

cols = ["Make", "Model", "Vehicle Class","Cylinders","Transmission","Fuel Type",
        "Fuel Consumption City (L/100 km)", "Fuel Consumption Hwy (L/100 km)",
        "Fuel Consumption Comb (L/100 km)","Fuel Consumption Comb (mpg)"]

cols_only_continues = ["Fuel Consumption City (L/100 km)", "Fuel Consumption Hwy (L/100 km)",
        "Fuel Consumption Comb (L/100 km)"]

hasher = FeatureHasher(outputCol="hashed_features", inputCols=cols_only_continues)
co2_data = hasher.transform(co2_data)

In [11]:
co2_data.select("hashed_features").show(5, truncate=False)

+---------------------------------------------+
|hashed_features                              |
+---------------------------------------------+
|(262144,[38607,109231,228390],[0.0,9.9,6.7]) |
|(262144,[38607,109231,228390],[0.0,11.2,7.7])|
|(262144,[38607,109231,228390],[0.0,6.0,5.8]) |
|(262144,[38607,109231,228390],[0.0,12.7,9.1])|
|(262144,[38607,109231,228390],[0.0,12.1,8.7])|
+---------------------------------------------+
only showing top 5 rows


25/10/14 18:30:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Fuel Type, Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km)
 Schema: Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (L/100 km)
Expected: Fuel Consumption City (L/100 km) but found: Fuel Type
CSV file: file:///Users/gafnts/Documents/Github/ml-with-spark/ml-with-spark/static/CO2_Emissions_Canada.csv


In [12]:
co2_data.select("hashed_features").take(1)

25/10/14 18:30:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Fuel Type, Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km)
 Schema: Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (L/100 km)
Expected: Fuel Consumption City (L/100 km) but found: Fuel Type
CSV file: file:///Users/gafnts/Documents/Github/ml-with-spark/ml-with-spark/static/CO2_Emissions_Canada.csv


[Row(hashed_features=SparseVector(262144, {38607: 0.0, 109231: 9.9, 228390: 6.7}))]

In [13]:
co2_data.select("hashed_features").show(5, truncate=False)

+---------------------------------------------+
|hashed_features                              |
+---------------------------------------------+
|(262144,[38607,109231,228390],[0.0,9.9,6.7]) |
|(262144,[38607,109231,228390],[0.0,11.2,7.7])|
|(262144,[38607,109231,228390],[0.0,6.0,5.8]) |
|(262144,[38607,109231,228390],[0.0,12.7,9.1])|
|(262144,[38607,109231,228390],[0.0,12.1,8.7])|
+---------------------------------------------+
only showing top 5 rows


25/10/14 18:30:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Fuel Type, Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km)
 Schema: Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (L/100 km)
Expected: Fuel Consumption City (L/100 km) but found: Fuel Type
CSV file: file:///Users/gafnts/Documents/Github/ml-with-spark/ml-with-spark/static/CO2_Emissions_Canada.csv


In [14]:
co2_data.printSchema()

root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Vehicle Class: string (nullable = true)
 |-- Cylinders: double (nullable = false)
 |-- Transmission: string (nullable = true)
 |-- Fuel Type: string (nullable = true)
 |-- Fuel Consumption City (L/100 km): double (nullable = false)
 |-- Fuel Consumption Hwy (L/100 km): double (nullable = false)
 |-- Fuel Consumption Comb (L/100 km): double (nullable = false)
 |-- Fuel Consumption Comb (mpg): double (nullable = false)
 |-- CO2: double (nullable = false)
 |-- hashed_features: vector (nullable = true)



# Time for selecting the most meaningful features:

In [15]:
from pyspark.ml.feature import UnivariateFeatureSelector

selector = UnivariateFeatureSelector(outputCol="selectedFeatures", featuresCol="hashed_features", labelCol="CO2")

selector.setFeatureType("continuous")
selector.setLabelType("continuous")

model = selector.fit(co2_data)
output = model.transform(co2_data)

25/10/14 18:31:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Fuel Type, Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (mpg)
 Schema: Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (L/100 km), CO2
Expected: Fuel Consumption City (L/100 km) but found: Fuel Type
CSV file: file:///Users/gafnts/Documents/Github/ml-with-spark/ml-with-spark/static/CO2_Emissions_Canada.csv
25/10/14 18:31:20 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB
25/10/14 18:31:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Fuel Type, Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (mpg)
 Schema: Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (L/100 km), CO2
Expected: Fuel Consumption City (L/100 km) but found: Fuel Type
CSV file: file:///Users/gafnts/Documents/Github/ml-w

In [16]:
output.select("selectedFeatures").show(5, truncate=False)

+-----------------------+
|selectedFeatures       |
+-----------------------+
|(50,[48,49],[9.9,6.7]) |
|(50,[48,49],[11.2,7.7])|
|(50,[48,49],[6.0,5.8]) |
|(50,[48,49],[12.7,9.1])|
|(50,[48,49],[12.1,8.7])|
+-----------------------+
only showing top 5 rows


25/10/14 18:31:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Fuel Type, Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km)
 Schema: Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (L/100 km)
Expected: Fuel Consumption City (L/100 km) but found: Fuel Type
CSV file: file:///Users/gafnts/Documents/Github/ml-with-spark/ml-with-spark/static/CO2_Emissions_Canada.csv


Count the number of available classes as it is needed as part of the last layer for the multilayer perceptron classifier.

Classifier trainer based on the Multilayer Perceptron. 
Each layer has sigmoid activation function, output layer has softmax. 

Number of inputs has to be equal to the size of feature vectors. 

Number of outputs has to be equal to the total number of labels.


Specify the layers for the neural network as follows: 

input layer => size 50 (features), two intermediate layers (i.e. hidden layer) of size 20 and 8 and output => size 70 as the largest value of CO2 is 69 and lables array is searched by index (classes).      




In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

mlp = MultilayerPerceptronClassifier(layers=[50,20, 8, 70], seed=123, featuresCol="selectedFeatures", labelCol="CO2")
mlp.setMaxIter(100)


model = mlp.fit(output)

25/10/14 18:32:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/10/14 18:32:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Fuel Type, Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (mpg)
 Schema: Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (L/100 km), CO2
Expected: Fuel Consumption City (L/100 km) but found: Fuel Type
CSV file: file:///Users/gafnts/Documents/Github/ml-with-spark/ml-with-spark/static/CO2_Emissions_Canada.csv
25/10/14 18:32:02 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
